In [55]:
import pandas as pd
import numpy as np
import chardet
import json
import pytz
import requests
import plotly.express as px
from dash import Dash, Input, Output
from dash import dcc
from dash import html
import dash
import plotly.graph_objects as go

#url = "https://drive.google.com/uc?export=download&id=1W-DbKPjq_zPX-pd6fNr-Vk9fDSy3f34i&authuser=1" #fechas responsables
url = "https://drive.google.com/uc?export=download&id=1JjTg4mXMo5zMICLiPIfMajW3IWv51oUI" #fuente.csv

df = pd.read_csv(url, sep=',')
df.head(4)

,Unnamed: 0,detalle,estado,titulo,cliente.nombre,fecha_server.$date,responsable.nombre,solucion.fecha_inicio,solucion.fecha_fin,solucion.medio.valor,solucion.detalle
0,0,se queda bloqueado y cuando funciona es muy lento,Cerrado,No puede trabajar en el programa,YAMOTOS QUILICHAO SA - YAMOTOS - CALLE 70,2016-11-05 09:16:27,Jennifer Valencia Esquivel,2016-11-05 09:16:33,2016-11-05 09:17:23,Internet-Remoto,se indexan los archivos y se soluciona el prob...
1,1,En contabilidad ya existe,Cerrado,No puede adicionar una NC,O´GRAFICAS IMPRESORES S.A.S,2016-11-05 09:49:42,Jennifer Valencia Esquivel,2016-11-05 09:49:46,2016-11-05 09:53:38,Internet-Remoto,se le indica que las notas credito por anulaci...
2,2,Dice que un ingeniero necesita soporte para el...,Cerrado,Solicita que atuendan llamada,FONDO DE EMPLEADOS DE INCAUCA COSECHA - FEINC...,2016-11-05 09:50:14,Lina Marcela Vargas Quintero,2016-11-05 09:57:37,2016-11-05 09:59:31,Llamada,nos comunicamos con el técnico informándole qu...
3,3,Dice que han tenido muchos problemas con el so...,Cerrado,Solicita soporte,YAMOTOS QUILICHAO SA - YAMOTOS - CALLE 70,2016-11-05 12:55:08,Jennifer Valencia Esquivel,2016-11-08 09:10:59,2016-11-08 09:11:21,Llamada,se resolvio en ticket con la señora Laura Camp...


In [18]:
"""columnas_a_eliminar = ['_id', 'departamento', 'fecha_creacion', 'prioridad', 'cliente._id', 'cliente.numero_identificacion', 'contacto.cargo', 'contacto.nombre', 'creador.cargo.__v', 'creador.cargo.consecutivo', 'creador.cargo.valor', 'creador.cargo.categoria', 'creador.cargo._id', 'creador.nombre', 'responsable._id', 'responsable.proceso._id', 'responsable.proceso.valor', 'responsable.tipo', 'responsable', 'responsable.proceso', 'creador.cargo', 'solucion.medio', 'solucion.medio._id', 'contacto', "tipo_soporte", "responsable.numero_identificacion","responsable.cargo"]
df = df.drop(columnas_a_eliminar, axis=1)"""

'columnas_a_eliminar = [\'_id\', \'departamento\', \'fecha_creacion\', \'prioridad\', \'cliente._id\', \'cliente.numero_identificacion\', \'contacto.cargo\', \'contacto.nombre\', \'creador.cargo.__v\', \'creador.cargo.consecutivo\', \'creador.cargo.valor\', \'creador.cargo.categoria\', \'creador.cargo._id\', \'creador.nombre\', \'responsable._id\', \'responsable.proceso._id\', \'responsable.proceso.valor\', \'responsable.tipo\', \'responsable\', \'responsable.proceso\', \'creador.cargo\', \'solucion.medio\', \'solucion.medio._id\', \'contacto\', "tipo_soporte", "responsable.numero_identificacion","responsable.cargo"]\ndf = df.drop(columnas_a_eliminar, axis=1)'

In [72]:
df['fecha_server.$date'] = pd.to_datetime(df['fecha_server.$date'])
df['YearMonth'] = df['fecha_server.$date'].dt.to_period('M')

In [56]:
df['responsable.nombre'] = df['responsable.nombre'].str.title()

df['responsable.nombre'].replace({
    'Santiago Ruiz Velasco': 'SANTIAGO RUIZ VELASCO',
    'Leydi Calad Blanco': 'LEIDY CALAD BLANCO'
}, inplace=True)

print(df['responsable.nombre'].unique())

['Jennifer Valencia Esquivel' 'Lina Marcela Vargas Quintero'
 'Bernardo Ruiz Caldas' 'Paul Andre Ruiz Caldas'
 'Silvana Irene Bonilla Llano' 'Leidy Calad Blanco'
 'SANTIAGO RUIZ VELASCO' 'Andres Felipe Vera Cifuentes'
 'Comptel System Undefined' 'Undefined Undefined' 'Luisa Maria Atehortua'
 'Jhonatan Galvis' 'Juliana Katherine Pedraza Cardenas'
 'Tivisay Rubiano Quintero' 'Miguel 0' 'Nathalia Rivera Posso'
 'Jean Carlos Quisoboni Jimenez' 'Maria Victoria Murillo Prado']


In [57]:
frecuencia_valores = df['responsable.nombre'].value_counts()
print(frecuencia_valores)

responsable.nombre
Jennifer Valencia Esquivel            16452
Luisa Maria Atehortua                  9058
Maria Victoria Murillo Prado           6242
Lina Marcela Vargas Quintero           5937
Leidy Calad Blanco                     4419
Juliana Katherine Pedraza Cardenas     1378
Jean Carlos Quisoboni Jimenez           673
Andres Felipe Vera Cifuentes            583
Paul Andre Ruiz Caldas                  370
Jhonatan Galvis                         289
Silvana Irene Bonilla Llano             132
SANTIAGO RUIZ VELASCO                   131
Undefined Undefined                      95
Bernardo Ruiz Caldas                     62
Miguel 0                                 36
Comptel System Undefined                  2
Tivisay Rubiano Quintero                  1
Nathalia Rivera Posso                     1
Name: count, dtype: int64


In [58]:
df = df.drop(df.index[46584:])

In [59]:
df = df.dropna()

In [60]:
print(df.isna().any())

Unnamed: 0               False
detalle                  False
estado                   False
titulo                   False
cliente.nombre           False
fecha_server.$date       False
responsable.nombre       False
solucion.fecha_inicio    False
solucion.fecha_fin       False
solucion.medio.valor     False
solucion.detalle         False
dtype: bool


In [61]:
df["TPR"] = (df["solucion.fecha_fin"] - df["solucion.fecha_inicio"])
df["TPPR"] = (df["solucion.fecha_inicio"] - df['fecha_server.$date'])

tpr_promedio_minutos = np.mean(df["TPR"].dt.total_seconds() / 60)
tppr_promedio_minutos = np.mean(df["TPPR"].dt.total_seconds() / 60)

tpr_promedio_str = pd.to_timedelta(tpr_promedio_minutos, unit='m')
tppr_promedio_str = pd.to_timedelta(tppr_promedio_minutos, unit='m')

print("Tiempo Promedio de Resolución (TPR):", tpr_promedio_str)
print("Tiempo Promedio de Primera Respuesta (TPPR):", tppr_promedio_str)

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [65]:
fig_bar_asesor = go.Figure(data=go.Bar(x=frecuencia_valores.index, y=frecuencia_valores.values, marker_color='rgba(152, 0, 0, .8)', name='Asesor'))
fig_bar_asesor.update_layout(title='Conteo de resolución de Tickets por Asesor', xaxis_title='Asesor', yaxis_title='Cantidad de Tickets')


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': 'rgba(152, 0, 0, .8)'},
              'name': 'Asesor',
              'type': 'bar',
              'x': array(['Jennifer Valencia Esquivel', 'Luisa Maria Atehortua',
                          'Maria Victoria Murillo Prado', 'Lina Marcela Vargas Quintero',
                          'Leidy Calad Blanco', 'Juliana Katherine Pedraza Cardenas',
                          'Jean Carlos Quisoboni Jimenez', 'Andres Felipe Vera Cifuentes',
                          'Paul Andre Ruiz Caldas', 'Jhonatan Galvis',
                          'Silvana Irene Bonilla Llano', 'SANTIAGO RUIZ VELASCO',
                          'Undefined Undefined', 'Bernardo Ruiz Caldas', 'Miguel 0',
                          'Comptel System Undefined', 'Tivisay Rubiano Quintero',
                          'Nathalia Rivera Posso'], dtype=object),
              'y': array([16452,  9058,  6242,  5937,  4419,  1378,   673,   583,   370,   289,
                            132,   131,    95,    62,    36,     2,     1,     1], dtype=int64)}],
    'layout': {'template': '...',
               'title': {'text': 'Conteo de resolución de Tickets por Asesor'},
               'xaxis': {'title': {'text': 'Asesor'}},
               'yaxis': {'title': {'text': 'Cantidad de Tickets'}}}
})

In [ ]:
fig_bar_asesor = go.Figure(data=go.Bar(x=frecuencia_valores.index, y=frecuencia_valores.values, marker_color='rgba(152, 0, 0, .8)', name='Asesor'))
fig_bar_asesor.update_layout(title='Conteo de resolución de Tickets por Asesor', xaxis_title='Asesor', yaxis_title='Cantidad de Tickets')

In [63]:
print(df.columns)

Index(['Unnamed: 0', 'detalle', 'estado', 'titulo', 'cliente.nombre',
       'fecha_server.$date', 'responsable.nombre', 'solucion.fecha_inicio',
       'solucion.fecha_fin', 'solucion.medio.valor', 'solucion.detalle'],
      dtype='object')


In [64]:
frec_canal = df['solucion.medio.valor'].value_counts()
print(frec_canal)

solucion.medio.valor
Internet-Remoto      40368
Llamada               3796
Orden de Servicio      918
Whatsapp               469
Visita                 137
Email                  111
Extranet                32
Skype                   17
Chat Portal              7
Carta                    1
Name: count, dtype: int64


In [76]:
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id='year-dropdown',
        options=[
            {'label': str(year), 'value': year} for year in df['fecha_server.$date'].dt.year.unique()
        ],
        value=df['fecha_server.$date'].dt.year.max(),
        style={'width': '50%'}
    ),
    dcc.Graph(id='fig_bar_tipo_soporte')
])

In [71]:
#DASH COMPLETE
"""
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id='graph-dropdown',
        options=[
            {'label': 'fig_bar_asesor Plot', 'value': 'fig_bar_asesor'},
            {'label': 'Bar Tipo Soporte Plot', 'value': 'fig_bar_tipo_soporte'},
        ],
        value='fig_bar_asesor'
    ),
    dcc.Graph(id='graph')
])

@app.callback(
    Output('graph', 'figure'),
    [Input('graph-dropdown', 'value')]
)
def update_graph(graph_choice):
    if graph_choice == 'fig_bar_asesor':
        return fig_bar_asesor
    elif graph_choice == 'fig_bar_tipo_soporte':
        return fig_bar_tipo_soporte

if __name__ == '__main__':
    app.run_server(debug=True)"""

"app = dash.Dash(__name__)\n\napp.layout = html.Div([\n    dcc.Dropdown(\n        id='graph-dropdown',\n        options=[\n            {'label': 'fig_bar_asesor Plot', 'value': 'fig_bar_asesor'},\n            {'label': 'Bar Tipo Soporte Plot', 'value': 'fig_bar_tipo_soporte'},\n        ],\n        value='fig_bar_asesor'\n    ),\n    dcc.Graph(id='graph')\n])\n\n@app.callback(\n    Output('graph', 'figure'),\n    [Input('graph-dropdown', 'value')]\n)\ndef update_graph(graph_choice):\n    if graph_choice == 'fig_bar_asesor':\n        return fig_bar_asesor\n    elif graph_choice == 'fig_bar_tipo_soporte':\n        return fig_bar_tipo_soporte\n\nif __name__ == '__main__':\n    app.run_server(debug=True)"

In [83]:
#ATTEMPT TO MAKE A CHART PER YEAR
@app.callback(
    Output('fig_bar_tipo_soporte', 'figure'),
    [Input('year-dropdown', 'value')]
)
def update_graph(selected_year):
    filtered_df = df[df['fecha_server.$date'].dt.year == selected_year]
    avg_tickets_by_month = filtered_df.groupby('YearMonth').size().mean(level=0)
    print(avg_tickets_by_month)
    
    fig_bar_tipo_soporte = go.Figure(data=go.Bar(x=avg_tickets_by_month.index, y=avg_tickets_by_month.values, marker_color='rgba(152, 0, 0, .8)', name='Tipo de Soporte'))
    fig_bar_tipo_soporte.update_layout(title=f'Conteo de Tickets por Tipo de Soporte en {selected_year}', xaxis_title='Tipo de Soporte', yaxis_title='Cantidad de Tickets')

    return fig_bar_tipo_soporte

if __name__ == '__main__':
    app.run_server()
    



NameError: name 'avg_tickets_by_month' is not defined